# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127343e+02     1.469504e+00
 * time: 0.07308197021484375
     1     1.112734e+01     8.914462e-01
 * time: 1.2357990741729736
     2    -1.200895e+01     1.035561e+00
 * time: 1.3343710899353027
     3    -3.423821e+01     7.161400e-01
 * time: 1.465433120727539
     4    -4.784928e+01     5.693378e-01
 * time: 1.5983860492706299
     5    -5.702086e+01     2.473440e-01
 * time: 1.7390949726104736
     6    -5.981411e+01     1.505911e-01
 * time: 1.8304870128631592
     7    -6.090430e+01     5.175327e-02
 * time: 1.9220671653747559
     8    -6.134062e+01     6.840021e-02
 * time: 2.0147721767425537
     9    -6.160221e+01     4.991552e-02
 * time: 2.1071231365203857
    10    -6.181179e+01     2.581028e-02
 * time: 2.213653087615967
    11    -6.196400e+01     1.880623e-02
 * time: 2.305546998977661
    12    -6.202980e+01     1.543435e-02
 * time: 2.397171974182129
    13    -6.208624e+01     1.448797e-02
 * time: 2.4897210597991

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557708
    AtomicNonlocal      14.8522662
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485385 
    Xc                  -19.3336826

    total               -62.261666459164
